In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)

plt.rc('font', family='AppleGothic') 
plt.rc('axes', unicode_minus=False)

warnings.filterwarnings('ignore')

In [2]:
show = pd.read_csv('../01.Data_Collection/show_timetable.csv')

In [3]:
show.shape

(13135726, 8)

In [4]:
show.head()

,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
0,0원,01관,11:00,2017-06-29,해빙,143.000000,롯데시네마 원주무실,디지털
1,0원,02관,10:00,2017-06-29,"당신, 거기 있어줄래요",119.000000,롯데시네마 원주무실,디지털
2,0원,03관,12:00,2017-06-29,동주,119.000000,롯데시네마 원주무실,디지털
3,0원,04관,17:00,2017-06-29,프리즌,144.000000,롯데시네마 원주무실,디지털
4,0원,05관,13:00,2017-06-29,터널,119.000000,롯데시네마 원주무실,디지털


In [5]:
# 전국 524개 영화관 대상
show['영화관'].nunique()

524

# 분석 영화 리스트에 있는데 상영내역에서 안 가져와진 영화 제목들 확인

In [6]:
show['영화명'].nunique()

662

In [7]:
film = pd.read_csv('../01.Data_Collection/films_final.csv')

In [8]:
len(film)

665

In [9]:
film[~film['영화명'].isin(show['영화명'])]['영화명']

227    인천상륙작전 : 익스텐디드 에디션(확장판)
332       드래곤빌리지<화이트니스타운을 구하라>
417               돌아와요 부산항애(愛)
Name: 영화명, dtype: object

In [11]:
set(film['영화명']) - set(show['영화명'])

{'돌아와요 부산항애(愛)', '드래곤빌리지<화이트니스타운을 구하라>', '인천상륙작전 : 익스텐디드 에디션(확장판)'}

In [ ]:
# film[~film['영화명'].isin(show_titles)].to_csv('film_add.csv', index=False)

# 영화별로 상영시각(시)와 총 공급 좌석수(좌석수*상영횟수) 비교

In [ ]:
show['상영시각_시'] = show['상영시각'].apply(lambda x: x.split(':')[0])

In [ ]:
show.head()

In [ ]:
scale = pd.DataFrame(show.groupby(['영화명', '상영시각_시'])['좌석수'].value_counts()).rename(columns={'좌석수':'횟수'}).reset_index()
scale.head(10)

In [ ]:
scale['총공급좌석수'] = scale['좌석수'] * scale['횟수']

In [ ]:
by_film = scale.groupby(['영화명', '상영시각_시'])['총공급좌석수'].sum().reset_index()
by_film.head()

In [ ]:
fig = px.line(by_film, x='상영시각_시', y='총공급좌석수', color='영화명', range_x=[6,28])
fig.show()

# 프라임시간대 총 공급 좌석수 합계로 클러스터링
전체 상영영화 수: 652개  
상영시간: 16~22시

In [ ]:
by_film['상영시각_시'] = by_film['상영시각_시'].astype(int)

In [ ]:
prime = by_film[(by_film['상영시각_시']>=16) & (by_film['상영시각_시']<=22)]

In [ ]:
prime.sample()

In [ ]:
prime = prime.groupby('영화명')['총공급좌석수'].sum().reset_index()
prime.sample()

In [ ]:
fig = px.histogram(prime, x="총공급좌석수")
fig.show()

In [ ]:
prime.describe()

In [ ]:
prime_500k = prime[prime['총공급좌석수']<500000]

fig = px.histogram(prime_500k, x="총공급좌석수", hover_name='영화명')
fig.show()

In [ ]:
prime_40k = prime[prime['총공급좌석수']<40000]

fig = px.histogram(prime_40k, x="총공급좌석수", hover_name='영화명')
fig.show()

In [ ]:
# 총 공급 좌석수 40000 미만 (~39999) : 1그룹
print(len(prime_40k))
prime_40k.sort_values('총공급좌석수', ascending=False)

In [ ]:
# 총 공급 좌석수 40000~459999 : 2그룹
print(len(prime[(prime['총공급좌석수']>=40000) & (prime['총공급좌석수']<460000)]))
prime[(prime['총공급좌석수']>=40000) & (prime['총공급좌석수']<460000)].sort_values('총공급좌석수', ascending=False)

In [ ]:
prime_700k = prime[(prime['총공급좌석수']>=460000) & (prime['총공급좌석수']<7000000)]

fig = px.histogram(prime_700k, x="총공급좌석수", hover_name='영화명')
fig.show()

In [ ]:
# 총 공급 좌석수 460000~3999999 : 3그룹
print(len(prime[(prime['총공급좌석수']>=460000) & (prime['총공급좌석수']<4000000)]))
prime[(prime['총공급좌석수']>=460000) & (prime['총공급좌석수']<4000000)].sort_values('총공급좌석수', ascending=False)

In [ ]:
# 총 공급 좌석수 4000000~6999999 : 4그룹
print(len(prime[(prime['총공급좌석수']>=4000000) & (prime['총공급좌석수']<7000000)]))
prime[(prime['총공급좌석수']>=4000000) & (prime['총공급좌석수']<7000000)].sort_values('총공급좌석수', ascending=False)

In [ ]:
# 총 공급 좌석수 7000000~15000000 : 5그룹
print(len(prime[prime['총공급좌석수']>=7000000]))
prime[prime['총공급좌석수']>=7000000].sort_values('총공급좌석수', ascending=False)

In [ ]:
len(prime)

1그룹 (239개): 총 공급 좌석수 0~39999   
2그룹 (195개) : 총 공급 좌석수 40000~459999  
3그룹 (154개) : 총 공급 좌석수 460000~3999999  
4그룹 (38개) : 총 공급 좌석수 4000000~6999999  
5그룹 (35개) : 총 공급 좌석수 7000000~15000000  
(아웃라이어: 극한직업 총 공급 좌석수 22523026)

In [ ]:
prime.to_csv('prime_new.csv', index=False)
# prime[prime['총공급좌석수']<40000].to_csv('prime_group1.csv', index=False)
prime[(prime['총공급좌석수']>=40000) & (prime['총공급좌석수']<460000)].to_csv('prime_group2_new.csv', index=False)
# prime[(prime['총공급좌석수']>=460000) & (prime['총공급좌석수']<4000000)].to_csv('prime_group3.csv', index=False)
# prime[(prime['총공급좌석수']>=4000000) & (prime['총공급좌석수']<7000000)].to_csv('prime_group4.csv', index=False)
prime[(prime['총공급좌석수']>=7000000) & (prime['총공급좌석수']<15000000)].to_csv('prime_group5_new.csv', index=False)

---
# 5그룹 EDA
총 공급 좌석수 7000000~15000000

In [ ]:
g5 = pd.read_csv('prime_group5_new.csv')
g5

In [ ]:
film = pd.read_csv('../01.Data_Collection/films_final.csv')
film.sample()

In [ ]:
screen = pd.read_csv('../01.Data_Collection/screens_final.csv')
screen.sample()

In [ ]:
g5 = g5.merge(film, how='left', on='영화명')
g5.sample()

In [ ]:
g5['F-rated'].hist()

총 공급 좌석수 7,500,001 이상인 5 그룹에 F-3인 영화가 하나도 없다.  
F-2 1개, F-1 4개, F-0 21개  

In [ ]:
len(group5[group5['F-rated']==0])

# 2그룹 EDA
총 공급 좌석수 40,001~321,000

In [ ]:
group2 = pd.read_csv('prime_group2.csv').merge(film, how='left', on='영화명')

In [ ]:
group2.corr()

In [ ]:
len(group2)

In [ ]:
group2['F-rated'].value_counts()

In [ ]:
rate = pd.read_csv('../01.Data_Collection/screen_show_seat_with_filtered_film.csv')
rate.sample()

In [ ]:
group2 = group2.merge(rate, how='left', on='영화명')
group2.sample()

In [ ]:
group2 = group2.drop(['개봉일_y', 'Unnamed: 0'], axis=1).rename(columns={'개봉일_x':'개봉일', '상영횟수_x':'상영횟수', '상영횟수_y':'일일상영횟수'})
group2.sample()

In [ ]:
group2['개봉n일차'] = group2['개봉n일차'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
group2['장르'] = group2['장르'].apply(lambda x: x.split(',')[0])

총공급좌석수와 전국관객수

In [ ]:
px.scatter(group2, x='총공급좌석수', y='전국관객수', color='F-rated', hover_name='영화명', trendline='ols')

In [ ]:
group2['장르'].unique()

총공급좌석수에 비해 전국관객수가 많은 영화들은 대부분 애니메이션이다.  
좌석판매율이 엄청 높겠지?

In [ ]:
group2['장르'].value_counts()

애니메이션 제외하고 보기

In [ ]:
group2_except_ani.corr()

In [ ]:
group2_except_ani['F-rated'].value_counts()

In [ ]:
px.scatter(group2_except_ani, x='총공급좌석수', y='전국관객수', color='F-rated', hover_name='영화명', trendline='ols')

벌새

In [ ]:
rate['개봉n일차'].dtype

In [ ]:
hummingbird = rate[rate['영화명']=='벌새']
hummingbird

In [ ]:
px.bar(hummingbird, x='개봉n일차', y='좌석판매율', range_x=[0,100])

In [ ]:
group2[group2['F-rated']==3]

In [ ]:
px.bar(rate[rate['영화명']=='소공녀'], x='개봉n일차', y='좌석판매율', color='스크린수', range_x=[0,150])

In [ ]:
data = rate[rate['영화명']=='소공녀']
plt.figure(figsize=[16, 6])
sns.barplot(x='개봉n일차', y='좌석판매율', data=data, color='black').set(xlim=(0,100))
sns.barplot(x='개봉n일차', y='스크린수', data=data).set(xlim=(0,100))
plt.show()